# Ensemble Learning

Import importent Library 

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder, StandardScaler


In [2]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

Read the CSV Data and Perform Basic Data Cleaning

In [11]:

# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

loan_amnt  int_rate  installment home_ownership  annual_inc  \
0    10500.0  0.001719       375.35           RENT     66000.0   
1    25000.0  0.002000       929.09       MORTGAGE    105000.0   
2    20000.0  0.002000       529.88       MORTGAGE     56000.0   
3    10000.0  0.001640       353.55           RENT     92000.0   
4    22000.0  0.001474       520.39       MORTGAGE     52000.0   

  verification_status   issue_d loan_status pymnt_plan    dti  ...  \
0     Source Verified  Mar-2019    low_risk          n  27.24  ...   
1            Verified  Mar-2019    low_risk          n  20.23  ...   
2            Verified  Mar-2019    low_risk          n  24.26  ...   
3            Verified  Mar-2019    low_risk          n  31.44  ...   
4        Not Verified  Mar-2019    low_risk          n  18.76  ...   

   pct_tl_nvr_dlq  percent_bc_gt_75  pub_rec_bankruptcies  tax_liens  \
0            85.7             100.0                   0.0        0.0   
1            91.2              50.0                   1.0        0.0   
2            66.7              50.0                   0.0        0.0   
3           100.0              50.0                   1.0        0.0   
4           100.0               0.0                   0.0        0.0   

   tot_hi_cred_lim  total_bal_ex_mort total_bc_limit  \
0          65687.0            38199.0         2000.0   
1         271427.0            60641.0        41200.0   
2          60644.0            45684.0         7500.0   
3          99506.0            68784.0        19700.0   
4         219750.0            25919.0        27600.0   

   total_il_high_credit_limit  hardship_flag  debt_settlement_flag  
0                     61987.0              N                     N  
1                     49197.0              N                     N  
2                     43144.0              N                     N  
3                     76506.0              N                     N  
4                     20000.0              N                     N  

[5 rows x 86 columns]

Split the Data into Training and Testing

In [12]:
# Create our features
X = df.drop(columns='loan_status',axis=1)
X=pd.get_dummies(X)

# Create our target
y = df['loan_status']

In [13]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

Data Pre-Processing

In [15]:
# I scale the Data
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

Train the model using the training data.

Calculate the balanced accuracy score from sklearn.metrics.

Display the confusion matrix from sklearn.metrics.

Generate a classication report using the imbalanced_classification_report from imbalanced-learn.

For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

Balanced Random Forest Classifier

In [17]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
random_forest = BalancedRandomForestClassifier(n_estimators = 100)
random_forest = random_forest.fit(X_train_scaled, y_train)

# Calculated the balanced accuracy score
y_pred_rf = random_forest.predict(X_test)
balanced_accuracy_score(y_test, y_pred_rf)
print(f"Banlanced accuracy score is {balanced_accuracy_score}")

# Display the confusion matrix
from sklearn.metrics import confusion_matrix
c_m = confusion_matrix(y_test, y_pred_rf)
print(f"Confusion Metric is {c_m}")

Banlanced accuracy score is <function balanced_accuracy_score at 0x0000025DC5E83670>
Confusion Metric is [[   51    40]
 [ 5221 11892]]


In [18]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.56      0.69      0.02      0.62      0.38        91
   low_risk       1.00      0.69      0.56      0.82      0.62      0.39     17113

avg / total       0.99      0.69      0.56      0.81      0.62      0.39     17204



In [20]:
 #List the features sorted in descending order by feature importance
import pandas as pd
pd.set_option('display.max_rows', 95)
feature_importances = pd.DataFrame(random_forest.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)

In [21]:
feature_importances.head(10)


importance
total_rec_prncp         0.074507
total_pymnt_inv         0.063221
total_rec_int           0.058302
last_pymnt_amnt         0.057473
total_pymnt             0.044982
int_rate                0.027057
dti                     0.018567
installment             0.018470
mo_sin_old_rev_tl_op    0.017981
issue_d_Jan-2019        0.017819

# Easy Ensemble Classifier

In [22]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
ee = EasyEnsembleClassifier(n_estimators=100,random_state=1)
ee.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [23]:
# Calculated the balanced accuracy score
y_pred_ee = ee.predict(X_test)
balanced_accuracy_score(y_test, y_pred_ee)

0.9301604140031067

In [24]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_ee)

array([[   85,     6],
       [ 1262, 15851]], dtype=int64)

In [25]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_ee))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.06      0.93      0.93      0.12      0.93      0.87        91
   low_risk       1.00      0.93      0.93      0.96      0.93      0.86     17113

avg / total       0.99      0.93      0.93      0.96      0.93      0.86     17204

